## Load dependencies

In [1]:
MODEL_CHECKPOINT_PATH = '/home/kvu/erc/data/audio weights/effnet_42.pth'

In [2]:
import sys
sys.path.append('/home/kvu/erc/libs')

import csv
import os
from sklearn.metrics import confusion_matrix
from tqdm import tqdm

import pandas
import torch
import yaml
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from PIL import Image
from torchvision import transforms as tvtf
from torch.utils.data import DataLoader
from utils.device import detach, move_to

from utils.getter import get_instance
from datasets import *

import warnings
warnings.filterwarnings("ignore")

## Load model weight and configs

In [3]:
config = torch.load(MODEL_CHECKPOINT_PATH)['config']
config

{'id': 'audio_effnet',
 'pretrained': None,
 'model': {'name': 'BaseTimmModel',
  'args': {'name': 'tf_efficientnet_b1_ap',
   'num_classes': 7,
   'freeze_backbone': False,
   'from_pretrained': True}},
 'optimizer': {'name': 'Adam', 'args': {'lr': 0.0003}},
 'loss': {'name': 'FocalLoss',
  'args': {'alpha': 0.75, 'gamma': 4, 'reduce': True}},
 'metric': [{'name': 'F1Weighted', 'args': {}}],
 'scheduler': {'name': 'ReduceLROnPlateau',
  'args': {'factor': 0.5, 'patience': 2}},
 'dataset': {'train': {'name': 'AudioDataset',
   'args': {'data_dir': '/content/src/data/MELD/raw-audios-wav/train/',
    'csv_path': '/content/src/data/MELD/raw-audios-wav/train.csv',
    'format': '.wav',
    'sr': 22050,
    'duration': 8,
    'n_mels': 128,
    'is_train': True},
   'loader': {'name': 'DataLoader',
    'args': {'batch_size': 8, 'shuffle': True, 'drop_last': True}}},
  'val': {'name': 'AudioDataset',
   'args': {'data_dir': '/content/src/data/MELD/raw-audios-wav/val/',
    'csv_path': '/cont

In [4]:
dev_id = 'cuda:{}'.format(config['gpus']) \
    if torch.cuda.is_available() and config.get('gpus', None) is not None \
    else 'cpu'

device = torch.device(dev_id)
model = get_instance(config['model']).to(device)
model.load_state_dict(torch.load(MODEL_CHECKPOINT_PATH)['model_state_dict'])

<All keys matched successfully>

## Load the dataset and dataloader

In [13]:
CSV_PATH = '/home/kvu/erc/data/raw-audios-wav/test.csv'
DATA_DIR = '/home/kvu/erc/data/raw-audios-wav/test/'

dataset = AudioDataset(csv_path=CSV_PATH, data_dir=DATA_DIR, format='.wav', is_train=False, sr=22050, duration=8, n_mels=128)
dataset_loader = DataLoader(dataset=dataset, batch_size=8, shuffle=False, pin_memory=False)

In [14]:
x, y, audio_id = dataset[0]
audio_id

'dia136_utt0'

## Feature extractor

In [7]:
model.model.reset_classifier(0)

In [15]:
for i in tqdm(range(len(dataset))):
    x, y, audio_id = dataset[i]
    x = torch.unsqueeze(x, 0)
    res = model(x.to(device).float())
    res = res.detach().cpu().numpy()
    np.save(os.path.join('/home/kvu/erc/Datasets/MELD/audio-features/test', audio_id + '.npy'), res)

100%|██████████| 2610/2610 [05:17<00:00,  8.22it/s]
